In [1]:
import os
import numpy as np
import pandas as pd
from typing import *

import joblib
import yaml
import json

import warnings

warnings.filterwarnings('ignore')

# Data import

In [2]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config['preprocessing']
training = config['train']
evaluate = config['evaluate']

# проверка столбцов с train
column_sequence_path = preproc['unique_values_path']
with open(column_sequence_path) as json_file:
    column_sequence = json.load(json_file)

In [3]:
df_test = pd.read_csv(evaluate['predict_path'])
df_test[:5]

,id,Section,Category,Type,Name,Manufactured,Price,Old_price,Fats,Carbohydrates,Squirrels
0,1106232,Бакалея,Крупы и макаронные изделия,Макароны и паста,"Макаронные изделия Farfalle 31 PastaZara, 500 г",Pasta Zara S.p.A.,82.99,107.99,1.2,71.00,12.00
1,1106602,Бакалея,"Соль, сахар, специи",Специи и приправы,"Розмарин Kamis, 10 г",McCormic Polska S.A.,46.99,46.99,15.0,21.00,5.00
2,1131710,Бакалея,Консервы и мёд,Овощные консервы,"Каперсы Italcarciofi в уксусе, 720 мл",Италкарчофи С.р.л.,387.99,477.99,0.1,6.48,1.91
3,124693,Бакалея,"Растительные масла, соусы",Растительные масла,Масло льняное Компас здоровья нерафинированное...,"ООО Научно-производственное обьединение ""Компа...",169.99,169.99,NaN,NaN,NaN
4,125093,"Хлеб, кондитерские изделия","Мармелад, зефир, пастила",Мармелад,Мармеладное драже Креззи-Джелли Ваш выбор с ар...,"ООО ""Русский кондитер""",28.99,28.99,0.1,90.00,3.00


# Preprocessing

In [4]:
def choose_fillna(data: pd.DataFrame) -> pd.DataFrame:
    """
    Заполнение пропусков в датасете, в зависимости от типа данных:
    'object' -> 'None',
    'int64' -> 999999,
    'float64' -> 999999,
    'bool' -> False,
    :param data: DataFrame;
    :return DataFrame.
    """
    for column in data.columns:
        if data[column].dtype == 'object':
            data[column].fillna('None', inplace=True)
        elif data[column].dtype == 'int64':
            data[column].fillna(999999, inplace=True)
        elif data[column].dtype == 'float64':
            data[column].fillna(999999, inplace=True)
        elif data[column].dtype == 'bool':
            data[column].fillna(False, inplace=True)
    return data


def transform_types(data: pd.DataFrame,
                    change_type_columns: dict) -> pd.DataFrame:
    """
    Преобразование признаков в заданный тип данных:
    :param data: DataFrame;
    :param change_type_columns: словарь с признаками и типами данных;
    :return: DataFrame.
    """
    return data.astype(change_type_columns, errors="raise")


def check_columns_evaluate(data: pd.DataFrame,
                           unique_values_path: str) -> pd.DataFrame:
    """
    Проверка на наличие признаков из train и упорядочивание признаков согласно train:
    :param data: test DataFrame;
    :param unique_values_path: путь до списка с признаками train, для сравнения;
    :return: test DataFrame.
    """
    with open(unique_values_path) as json_file:
        unique_values = json.load(json_file)

    column_sequence = unique_values.keys()

    assert set(column_sequence) == set(data.columns), 'Разные признаки'
    return data[column_sequence]

In [5]:
def pipeline_preprocess(data: pd.DataFrame,
                        flg_evaluate: bool = True,
                        **kwargs) -> pd.DataFrame:
    """
    Пайплайн по предобработке данных:
    :param data: DataFrame;
    :param flg_evaluate: флаг для evaluate;
    :return: DataFrame.
    """
    # удаление неиспользуемых колонок
    data = data.drop(kwargs['drop_columns'], axis=1, errors='ignore')
    # проверка DataFrame на совпадение с признаками из train
    # либо сохранение уникальных данных с признаками из train
    if flg_evaluate:
        data = check_columns_evaluate(
            data=data, unique_values_path=kwargs['unique_values_path'])
    else:
        save_unique_train_data(
            data=data,
            drop_columns=kwargs['drop_columns'],
            target_column=kwargs['target_column'],
            unique_values_path=kwargs['unique_values_path'],
        )

    # заполнение пропусков
    data = choose_fillna(data)

    # замена типов данных
    data = transform_types(
        data=data, change_type_columns=kwargs['change_type_columns_test'])
    return data

In [6]:
data_proc_test = pipeline_preprocess(data=df_test, **preproc)

# Evaluate

In [7]:
model = joblib.load(training['model_path'])
data_proc_test['predict'] = model.predict(data_proc_test)

In [8]:
data_proc_test

,Section,Category,Type,Manufactured,Price,predict
0,Бакалея,Крупы и макаронные изделия,Макароны и паста,Pasta Zara S.p.A.,82,352.675843
1,Бакалея,"Соль, сахар, специи",Специи и приправы,McCormic Polska S.A.,46,247.151956
2,Бакалея,Консервы и мёд,Овощные консервы,Италкарчофи С.р.л.,387,88.375419
3,Бакалея,"Растительные масла, соусы",Растительные масла,"ООО Научно-производственное обьединение ""Компа...",169,898.292469
4,"Хлеб, кондитерские изделия","Мармелад, зефир, пастила",Мармелад,"ООО ""Русский кондитер""",28,331.745103
5,Напитки,Морсы и компоты,None,ООО «Производственный Холдинг «Меркурий»,264,49.999731
6,Бакалея,"Растительные масла, соусы","Соусы, кетчупы, томатные пасты","ООО ""Марс""",122,48.683304
7,"Молочные продукты, сыр, яйца",Сыры,Сыры с плесенью,"ООО ""Калория""",1739,338.837791
8,Бакалея,Консервы и мёд,Мёд,"ООО ""Горячеключевская Пчеловодная Компания""",294,326.589640
9,"Хлеб, кондитерские изделия","Печенье, пряники, вафли","Печенье, галеты, крекеры","ООО ""Производственная компания ""Акульчев""",78,381.242546
